In [44]:
import pandas as pd
from sklearn.utils import resample
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from sklearn.impute import SimpleImputer


In [45]:
#load dataset
df = pd.read_csv("../data/processed/home_credit_train_ready.csv")

In [46]:
#rebalance dataset
class_0 = df[df['TARGET'] == 0]
class_1 = df[df['TARGET'] == 1]

# balance mayority class with minority
sub_class = resample(class_0, replace=False, n_samples=len(class_1), random_state=42)

# combine
df = pd.concat([sub_class, class_1])

In [47]:
y = df['TARGET']
X = df.drop(columns=['TARGET', 'SK_ID_CURR'])  #adde sk_id_curr

In [48]:
#manages divided by 0 values
X.replace([np.inf, -np.inf], np.nan, inplace=True)

In [49]:
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

num_imputer = SimpleImputer(strategy="median")  #change median for mean
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

In [50]:
for col in X.select_dtypes("object"):
    X[col] = X[col].astype("category")

In [51]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y ,random_state=42)

In [52]:
#catboost features and kfold
cat_features = [
    col for col in X.columns
    if X[col].dtype == 'category'
]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
aucs = []

for fold, (tr, va) in enumerate(skf.split(X, y), 1):
    model = CatBoostClassifier(
        iterations=4000,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3,
        loss_function='Logloss',
        eval_metric='AUC',
        random_seed=42,
        early_stopping_rounds=300,
        verbose=200,
    )

    model.fit(
        X.iloc[tr], y.iloc[tr],
        eval_set=(X.iloc[va], y.iloc[va]),
        cat_features=cat_features
    )

    preds = model.predict_proba(X.iloc[va])[:, 1]
    auc = roc_auc_score(y.iloc[va], preds)
    aucs.append(auc)

    print(f'Fold {fold} AUC: {auc:.5f}')

print(f'\nAUC medio: {np.mean(aucs):.5f} ± {np.std(aucs):.5f}')



0:	test: 0.7188045	best: 0.7188045 (0)	total: 93.2ms	remaining: 6m 12s
200:	test: 0.7663852	best: 0.7663852 (200)	total: 21.1s	remaining: 6m 39s
400:	test: 0.7727573	best: 0.7727644 (399)	total: 41.9s	remaining: 6m 16s
600:	test: 0.7755787	best: 0.7755787 (600)	total: 1m 2s	remaining: 5m 54s
800:	test: 0.7761829	best: 0.7762892 (788)	total: 1m 23s	remaining: 5m 32s
1000:	test: 0.7769026	best: 0.7769136 (998)	total: 1m 48s	remaining: 5m 24s
1200:	test: 0.7776731	best: 0.7776991 (1193)	total: 2m 22s	remaining: 5m 31s
1400:	test: 0.7782719	best: 0.7783014 (1399)	total: 2m 45s	remaining: 5m 7s
1600:	test: 0.7782065	best: 0.7783401 (1502)	total: 3m 6s	remaining: 4m 39s
1800:	test: 0.7781525	best: 0.7783401 (1502)	total: 3m 27s	remaining: 4m 12s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.7783401009
bestIteration = 1502

Shrink model to first 1503 iterations.
Fold 1 AUC: 0.77834
0:	test: 0.7293140	best: 0.7293140 (0)	total: 95.4ms	remaining: 6m 21s
200:	test: 0.77309

In [53]:

# Probabilidades (idéntico conceptualmente)
y_val_proba = model.predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

#optimal threshold
thresholds = np.linspace(0.05, 0.6, 100)
f1_scores = [f1_score(y_val, y_val_proba >= t) for t in thresholds]

optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f'optimal threshold: {optimal_threshold}')

#Threshold (solo para métricas de clasificación)
adj_threshold = optimal_threshold

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

cm = confusion_matrix(y_val, y_val_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0, 0]:6d}   {cm[0, 1]:6d}")
print(f"Real Default   {cm[1, 0]:6d}   {cm[1, 1]:6d}")

optimal threshold: 0.4333333333333333
FPR: 0.3205
Precision (PPV): 0.7285
AUC validación: 0.85443
              precision    recall  f1-score   support

           0       0.83      0.68      0.75      2184
           1       0.73      0.86      0.79      2183

    accuracy                           0.77      4367
   macro avg       0.78      0.77      0.77      4367
weighted avg       0.78      0.77      0.77      4367


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def      1484      700
Real Default      305     1878


In [57]:
df = pd.read_csv("../data/processed/home_credit_train_ready.csv")

y = df['TARGET']
X = df.drop(columns = ['TARGET', 'SK_ID_CURR'])

X.replace([np.inf, -np.inf], np.nan, inplace=True)

cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

num_imputer = SimpleImputer(strategy="median")  #change median for mean
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y ,random_state=42)

# Probabilidades (idéntico conceptualmente)
y_val_proba = model.predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

#optimal threshold
thresholds = np.linspace(0.05, 0.6, 100)
f1_scores = [f1_score(y_val, y_val_proba >= t) for t in thresholds]

optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f'optimal threshold: {optimal_threshold}')

#Threshold (solo para métricas de clasificación)
adj_threshold = optimal_threshold

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

cm = confusion_matrix(y_val, y_val_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0, 0]:6d}   {cm[0, 1]:6d}")
print(f"Real Default   {cm[1, 0]:6d}   {cm[1, 1]:6d}")


optimal threshold: 0.6
FPR: 0.1676
Precision (PPV): 0.2568
AUC validación: 0.81339
              precision    recall  f1-score   support

           0       0.96      0.83      0.89     23030
           1       0.26      0.61      0.36      2184

    accuracy                           0.81     25214
   macro avg       0.61      0.72      0.63     25214
weighted avg       0.90      0.81      0.84     25214


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def     19170     3860
Real Default      850     1334
